In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, random, time

import warnings
warnings.filterwarnings('ignore')

In [18]:
starting_dir = os.getcwd()
print ("starting_dir:" + str(starting_dir))

# mac - phase I & II - morpho
#complete_file_path = '/Users/kimd999/research/projects/toxicity/per_each_data/phase_I_II/input/morpho/tall/2_after_fixing_MO24/final_full/zf_morphology_data_335_chemicals_2020DEC16_fixed.csv'
# 335 chemicals

# mac - phase III - morpho - 23 endpoints - full
#complete_file_path= '/Users/kimd999/research/projects/toxicity/per_each_data/phase_III/input/morpho/full/tall/Tanguay_Phase_3_zf_morphology_data_PNNL_2021MAR23_full_w_23_endpoints.csv'
# 118 chemicals

# mac - phase III - morpho - 14 endpoints - full
complete_file_path = '/Users/kimd999/research/projects/toxicity/per_each_data/phase_III/input/morpho/full/tall/Tanguay_Phase_3_zf_morphology_data_PNNL_2021MAR23_full_w_14_endpoints.csv'

df_morph = pd.read_csv(complete_file_path, header = 0)
pd.set_option('display.max_columns', None)

df_morph.head()

starting_dir:/Users/kimd999/research/script_not_in_dropbox/srpAnalytics/analysis/latest/1_reformat/all_targets/morpho


,chemical.id,conc,bottle.id,plate.id,well,endpoint,value
0,725,0.0,C331561,21511,A01,MO24,0.0
1,725,0.0,C331561,21511,A02,MO24,0.0
2,725,0.0,C331561,21511,A03,MO24,0.0
3,725,0.0,C331561,21511,A04,MO24,0.0
4,725,0.0,C331561,21511,A05,MO24,0.0


In [7]:
# Keep only relevant columns
columns_to_keep = ['chemical.id', 'conc', 'bottle.id', 'plate.id', 'well', 'endpoint', 'value'] 
# 'bottle.id' is added since Lisa wants to use it to join with behavioral data

df_select = df_morph.loc[:,columns_to_keep]
df_select.head()
df_select.tail()

,chemical.id,conc,bottle.id,plate.id,well,endpoint,value
326951,4457,100.0,C665534,22097,D01,DNC_,0.0
326952,4457,100.0,C665534,22097,E01,DNC_,0.0
326953,4457,100.0,C665534,22097,F01,DNC_,0.0
326954,4457,100.0,C665534,22097,G01,DNC_,0.0
326955,4457,100.0,C665534,22097,H01,DNC_,0.0


In [8]:
print ("<before dropping duplicates> len(df_select):" + str(len(df_select)))
df_select = df_select.drop_duplicates()
print ("<after dropping duplicates>  len(df_select):" + str(len(df_select)))

<before dropping duplicates> len(df_select):326956
<after dropping duplicates>  len(df_select):326956


In [ ]:
# tall -> wide
start_time = time.time()

df_reformatted = pd.DataFrame()

full_devel = "full"
#full_devel = "devel"

chemical_id_from_here = np.unique(df_select['chemical.id'])

if (full_devel == "devel"):
    randomly_chosen = random.sample(set(chemical_id_from_here), 1)
    chemical_id_from_here = []
    for i in range(len(randomly_chosen)):
        chemical_id_from_here.append(randomly_chosen[i])

if (os.path.isfile("cannot_process.txt") == True):
    os.remove("cannot_process.txt")

total_number_of_chemicals_to_processed = len(chemical_id_from_here)
number_of_chemicals_processed = 0

for chemical_index in chemical_id_from_here:
    print("\nchemical_index:" + str(chemical_index))        
    
    df_per_chemical = df_select.loc[df_morph['chemical.id'] == chemical_index,:]
    #morph_data_chemical.to_csv('morph_data_chemical_1694.csv', index=False)
#    display(morph_data_chemical.head())
    
    # Append chemical_plate_well as a unique identifier
    df_per_chemical.insert(0, 'chemical_plate_well', df_per_chemical.loc[:,['chemical.id','plate.id', 'well']].apply(lambda x: '_'.join(x.map(str)), axis = 1))

    #print ("len(morph_data_chemical.chemical_plate_well):" + str(len(morph_data_chemical.chemical_plate_well)))    
    for cpw in np.unique(df_per_chemical.chemical_plate_well):
        df_per_cpw = df_per_chemical.loc[df_per_chemical.chemical_plate_well == cpw,:]
        #df_per_cpw_filename = 'df_per_cpw_' + str(chemical_index) + '.csv'
        #df_per_cpw.to_csv(df_per_cpw_filename, index=False)
        
        if (len(np.unique(df_per_cpw['conc'])) != 1):
            print ("can't process" + str(chemical_index) + " chemical")
            f = open("cannot_process.txt", "a")
            f.write(str(chemical_index)+"\n")
            f.close()
            break
        
        try:
            plate_id = pd.to_numeric(df_per_cpw['plate.id'])
        except:
            # we need this to deal exceptional case like "plate.id = TP967-E9-P1 for chemical 163 in phase I,II"
            plate_id = df_per_cpw['plate.id']

        if(len(df_per_cpw.endpoint) == 14):
            concat_this = pd.DataFrame(
                {
                    'chemical.id': np.unique(df_per_cpw['chemical.id']),
                    'plate.id': np.unique(plate_id),
                    'well': np.unique(df_per_cpw['well']),
                    'chemical_plate_well': np.unique(df_per_cpw['chemical_plate_well']),
                    'conc': np.unique(df_per_cpw['conc']),
                    'AXIS': df_per_cpw.value[df_per_cpw.endpoint == 'AXIS'].values,
                    'BRN_': df_per_cpw.value[df_per_cpw.endpoint == 'BRN_'].values,
                    'CRAN': df_per_cpw.value[df_per_cpw.endpoint == 'CRAN'].values,
                    'DNC_': df_per_cpw.value[df_per_cpw.endpoint == 'DNC_'].values,
                    'DP24': df_per_cpw.value[df_per_cpw.endpoint == 'DP24'].values,
                    'EDEM': df_per_cpw.value[df_per_cpw.endpoint == 'EDEM'].values,
                    'LTRK': df_per_cpw.value[df_per_cpw.endpoint == 'LTRK'].values,
                    'MO24': df_per_cpw.value[df_per_cpw.endpoint == 'MO24'].values,
                    'MORT': df_per_cpw.value[df_per_cpw.endpoint == 'MORT'].values,
                    'MUSC': df_per_cpw.value[df_per_cpw.endpoint == 'MUSC'].values,
                    'NC__': df_per_cpw.value[df_per_cpw.endpoint == 'NC__'].values,
                    'SKIN': df_per_cpw.value[df_per_cpw.endpoint == 'SKIN'].values,
                    'SM24': df_per_cpw.value[df_per_cpw.endpoint == 'SM24'].values,
                    'TCHR': df_per_cpw.value[df_per_cpw.endpoint == 'TCHR'].values,
                })
            df_reformatted = pd.concat([df_reformatted, concat_this])
        elif(len(df_per_cpw.endpoint) == 23):
            concat_this = pd.DataFrame(
                {
                    'chemical.id':         np.unique(df_per_cpw['chemical.id']),
                    'plate.id':            np.unique(plate_id),
                    'well':                np.unique(df_per_cpw['well']),
                    'chemical_plate_well': np.unique(df_per_cpw['chemical_plate_well']),
                    'conc':                np.unique(df_per_cpw['conc']),
                    'AXIS': df_per_cpw.value[df_per_cpw.endpoint == 'AXIS'].values,
                    'BRAI': df_per_cpw.value[df_per_cpw.endpoint == 'BRAI'].values,
                    'CFIN': df_per_cpw.value[df_per_cpw.endpoint == 'CFIN'].values,
                    'CIRC': df_per_cpw.value[df_per_cpw.endpoint == 'CIRC'].values,
                    'DNC_': df_per_cpw.value[df_per_cpw.endpoint == 'DNC_'].values,
                    'DP24': df_per_cpw.value[df_per_cpw.endpoint == 'DP24'].values,
                    'EYE_': df_per_cpw.value[df_per_cpw.endpoint == 'EYE_'].values,
                    'JAW_': df_per_cpw.value[df_per_cpw.endpoint == 'JAW_'].values,
                    'MO24': df_per_cpw.value[df_per_cpw.endpoint == 'MO24'].values,
                    'MORT': df_per_cpw.value[df_per_cpw.endpoint == 'MORT'].values,
                    'NC24': df_per_cpw.value[df_per_cpw.endpoint == 'NC24'].values,
                    'NC__': df_per_cpw.value[df_per_cpw.endpoint == 'NC__'].values,
                    'OTIC': df_per_cpw.value[df_per_cpw.endpoint == 'OTIC'].values,
                    'PE__': df_per_cpw.value[df_per_cpw.endpoint == 'PE__'].values,
                    'PFIN': df_per_cpw.value[df_per_cpw.endpoint == 'PFIN'].values,
                    'PIG_': df_per_cpw.value[df_per_cpw.endpoint == 'PIG_'].values,
                    'SM24': df_per_cpw.value[df_per_cpw.endpoint == 'SM24'].values,
                    'SNOU': df_per_cpw.value[df_per_cpw.endpoint == 'SNOU'].values,
                    'SOMI': df_per_cpw.value[df_per_cpw.endpoint == 'SOMI'].values,
                    'SWIM': df_per_cpw.value[df_per_cpw.endpoint == 'SWIM'].values,
                    'TRUN': df_per_cpw.value[df_per_cpw.endpoint == 'TRUN'].values,
                    'TR__': df_per_cpw.value[df_per_cpw.endpoint == 'TR__'].values,
                    'YSE_': df_per_cpw.value[df_per_cpw.endpoint == 'YSE_'].values,
                })
            df_reformatted = pd.concat([df_reformatted, concat_this])
        elif(len(df_per_cpw.endpoint) == 30):
            concat_this = pd.DataFrame(
                {
                    'chemical.id': np.unique(df_per_cpw['chemical.id']),
                    'plate.id': np.unique(plate_id),
                    'well': np.unique(df_per_cpw['well']),
                    'chemical_plate_well': np.unique(df_per_cpw['chemical_plate_well']),
                    'conc': np.unique(df_per_cpw['conc']),
                    'AXIS': df_per_cpw.value[df_per_cpw.endpoint == 'AXIS'].values,
                    'BRAI': df_per_cpw.value[df_per_cpw.endpoint == 'BRAI'].values,
                    'BRN_': df_per_cpw.value[df_per_cpw.endpoint == 'BRN_'].values,
                    'CFIN': df_per_cpw.value[df_per_cpw.endpoint == 'CFIN'].values,
                    'CIRC': df_per_cpw.value[df_per_cpw.endpoint == 'CIRC'].values,
                    'CRAN': df_per_cpw.value[df_per_cpw.endpoint == 'CRAN'].values,
                    'DNC_': df_per_cpw.value[df_per_cpw.endpoint == 'DNC_'].values,
                    'DP24': df_per_cpw.value[df_per_cpw.endpoint == 'DP24'].values,
                    'EDEM': df_per_cpw.value[df_per_cpw.endpoint == 'EDEM'].values,
                    'EYE_': df_per_cpw.value[df_per_cpw.endpoint == 'EYE_'].values,
                    'JAW_': df_per_cpw.value[df_per_cpw.endpoint == 'JAW_'].values,
                    'LTRK': df_per_cpw.value[df_per_cpw.endpoint == 'LTRK'].values,
                    'MO24': df_per_cpw.value[df_per_cpw.endpoint == 'MO24'].values,
                    'MORT': df_per_cpw.value[df_per_cpw.endpoint == 'MORT'].values,
                    'MUSC': df_per_cpw.value[df_per_cpw.endpoint == 'MUSC'].values,
                    'NC24': df_per_cpw.value[df_per_cpw.endpoint == 'NC24'].values,
                    'NC__': df_per_cpw.value[df_per_cpw.endpoint == 'NC__'].values,
                    'OTIC': df_per_cpw.value[df_per_cpw.endpoint == 'OTIC'].values,
                    'PE__': df_per_cpw.value[df_per_cpw.endpoint == 'PE__'].values,
                    'PFIN': df_per_cpw.value[df_per_cpw.endpoint == 'PFIN'].values,
                    'PIG_': df_per_cpw.value[df_per_cpw.endpoint == 'PIG_'].values,
                    'SM24': df_per_cpw.value[df_per_cpw.endpoint == 'SM24'].values,
                    'SNOU': df_per_cpw.value[df_per_cpw.endpoint == 'SNOU'].values,
                    'SOMI': df_per_cpw.value[df_per_cpw.endpoint == 'SOMI'].values,
                    'SWIM': df_per_cpw.value[df_per_cpw.endpoint == 'SWIM'].values,
                    'TCHR': df_per_cpw.value[df_per_cpw.endpoint == 'TCHR'].values,
                    'TRUN': df_per_cpw.value[df_per_cpw.endpoint == 'TRUN'].values,
                    'TR__': df_per_cpw.value[df_per_cpw.endpoint == 'TR__'].values,
                    'YSE_': df_per_cpw.value[df_per_cpw.endpoint == 'YSE_'].values,
                })
            df_reformatted = pd.concat([df_reformatted, concat_this])
        else:
            print ("len(group.endpoint):" + str(len(group.endpoint)))
            a=b
    number_of_chemicals_processed += 1
    print_this = str(number_of_chemicals_processed) + " chemicals processed out of " + str(total_number_of_chemicals_to_processed)
    print(print_this)
    
####### <begin> possible manual update of entries
find_this = df_reformatted.loc[(df_reformatted['chemical.id'] == 746) \
                & (df_reformatted['plate.id'] == 12953) \
                & (df_reformatted['well'] == "E03"), 'MORT'] 
if (len(find_this) != 0):
    print ("for phase I, II, automatically fix small manual mistakes")
    df_reformatted.loc[(df_reformatted['chemical.id'] == 746) \
                & (df_reformatted['plate.id'] == 12953) \
                & (df_reformatted['well'] == "E03"), 'MORT'] = 0

find_this = df_reformatted.loc[(df_reformatted['chemical.id'] == 1171) \
                    & (df_reformatted['plate.id'] == 15205) \
                    & (df_reformatted['well'] == "D02"), 'MORT']
if (len(find_this) != 0):
    print ("for phase I, II, automatically fix small manual mistakes")
    df_reformatted.loc[(df_reformatted['chemical.id'] == 1171) \
                    & (df_reformatted['plate.id'] == 15205) \
                    & (df_reformatted['well'] == "D02"), 'MORT'] = 0
####### <end> possible manual update of entries

end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("tall -> wide done, it took:"+str(time_took)) 
# took  3 minutes for 139 chemicals
# took 13 minutes for 344 chemicals

In [13]:
display(df_reformatted.head())
#display(df_reformatted.tail())
display(df_reformatted.shape)

,chemical.id,plate.id,well,chemical_plate_well,conc,AXIS,BRN_,CRAN,DNC_,DP24,EDEM,LTRK,MO24,MORT,MUSC,NC__,SKIN,SM24,TCHR
0,725,21511,A01,725_21511_A01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,725,21511,A02,725_21511_A02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,725,21511,A03,725_21511_A03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,725,21511,A04,725_21511_A04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,725,21511,A05,725_21511_A05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


(23354, 19)

In [15]:
df_reformatted_DNC_0 = pd.DataFrame()
df_reformatted_DNC_0 = df_reformatted.loc[df_reformatted['DNC_'] == 0.0]
print ("df_reformatted_DNC_0.shape:" + str(df_reformatted_DNC_0.shape))

df_reformatted_DNC_0.shape:(23330, 19)


In [16]:
output_complete_file_path = complete_file_path[:-4] + "_wide_DNC_0_" \
                            + str(full_devel) + ".csv"
display ('output_complete_file_path:', str(output_complete_file_path))

df_reformatted_DNC_0.to_csv(output_complete_file_path,index=False)

'output_complete_file_path:'

'/Users/kimd999/research/projects/toxicity/per_each_data/phase_III/input/morpho/full/tall/Tanguay_Phase_3_zf_morphology_data_PNNL_2021MAR23_full_w_14_endpoints_wide_DNC_0_full.csv'